# **ladatito**
O bot assistente da LADATA (Liga Acadêmica de Ciência de Dados).

### **Equipe**
Nayla Sahra Santos das Chagas
> nayla.chagas@dcomp.ufs.br <br>
ORCID: https://orcid.org/0009-0004-8268-5484



Túlio Sousa de Gois
> tulio.gois@dcomp.ufs.br<br>
ORCID: https://orcid.org/0009-0000-5270-8033

## **Recursos e requisitos**


O ladatito foi desenvolvido para ser um chatbot para o Discord, dessa forma, algumas bibliotecas não serão necessárias para a sua execução no Colab.
Assim, a seção que trata das bibliotecas será divididas em duas partes:
- Bibliotecas (Gerais);
- Biblitoecas (Discord).

---

* **Modelo**<br>
`gemini-pro`   - [Link para API](https://ai.google.dev/models/gemini?hl=pt-br)
* **Bibliotecas (Gerais)**<br>
    - [google-generativeai](https://github.com/google/generative-ai-python)<br>
    `pip install google-generativeai`
    - [chromadb](https://github.com/chroma-core/chroma)<br>
    `pip install chromadb`
    - [pypdf](https://pypdf.readthedocs.io/en/stable/)<br>
    `pip install pypdf`
* **Bibliotecas (Discord)**<br>
    - [discord](https://discordpy.readthedocs.io/en/stable/)<br>
    `pip install discord`
    - [aiohttp](https://docs.aiohttp.org/en/stable/)<br>
    `pip install aiohttp`
* **Documentos (RAG)**
    - [Projeto da LADATA](https://drive.google.com/file/d/15Uk5I3y6y0mRbOi21LBb4oPXUcUH2T3r/view?usp=sharing)
    - [Estatuto da LADATA](https://drive.google.com/file/d/1W7WDEYyxLbEJpBI0CmpLu9cfNGghVt2R/view?usp=sharing)



**! Atenção !**<br> _Para o correto funcionamento do RAG é necessário baixar os documentos e fazer o upload para o Google Colab_

---

# **ladatito**
> versão _Google Colab_

## **Configurações básicas**

### **Instalação das bibliotecas**

In [ ]:
!pip install --quiet google-generativeai
!pip install --quiet chromadb
!pip install --quiet pypdf

### **imports**

In [ ]:
import re
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from pypdf import PdfReader
from typing import List
import google.generativeai as genai

#armazenar chave da API do Colab
from google.colab import userdata

#bibliotecas necessárias para exibição
import textwrap
from IPython.display import display
from IPython.display import Markdown

### **Conexão com a API do Google**

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## **Classes e métodos do ladatito**

### **Geração de embeddings**

In [15]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = "models/embedding-001"
        title = "Custom query"
        return genai.embed_content(model=model,
                                   content=input,
                                   task_type="retrieval_document",
                                   title=title)["embedding"]

### **Processamento dos Documentos para RAG**

In [16]:
class EnviromentLADATITO():
    def __init__(self):
        self.key = ""
        self.arquivos = "/content/Estatuto da Liga Acadêmica de Ciência de Dados (LADATA).pdf"
        self.db_path = "ladatito_db"
        self.name = "ladatito"

    def load_pdf(self):
        reader = PdfReader(self.arquivos)

        text = ""
        for page in reader.pages:
            text += page.extract_text()

        return text

    def split_text(self):

        text = self.load_pdf()
        split_text = re.split('/n /n', text)
        return [i for i in split_text if i != ""]

    def create_chroma_db(self):

        chroma_client = chromadb.PersistentClient(path= self.db_path)
        db = chroma_client.create_collection(name= self.name, embedding_function=GeminiEmbeddingFunction())

        for i, d in enumerate(self.split_text()):
            db.add(documents=d, ids=str(i))

        return db, self.name

    def load_chroma_collection(self):
        chroma_client = chromadb.PersistentClient(path = self.db_path)
        db = chroma_client.get_collection(name = self.name, embedding_function=GeminiEmbeddingFunction())

        return db

### **Classe geradora do ladatito**

In [17]:
class GenerateLADATITO(EnviromentLADATITO):

    def __init__(self):
        self.enviroment = EnviromentLADATITO().create_chroma_db()
        self.db = EnviromentLADATITO().load_chroma_collection()
        self.key = GOOGLE_API_KEY

    def get_relevant_passage(self, query):
        passage = self.db.query(query_texts=[query], n_results=100)['documents'][0]
        return passage


    def make_rag_prompt(self, query, relevant_passage):
        escaped = relevant_passage.replace("'", "").replace('"', "").replace("/n", " ")
        prompt = ("""Você é um bot útil e informativo, chamado Ladatito, que responde a perguntas sobre a LADATA usando o texto da passagem de referência incluída abaixo. /
        Certifique-se de responder em uma frase completa, sendo abrangente, incluindo todas as informações básicas relevantes. /
        No entanto, você está falando para um público não técnico, portanto, analise conceitos complicados e/ou
        adote um tom amigável e descontraído, de modo a sempre cumprimentar seu público de forma calorosa. Caso queiram saber quem você é, identifique-se como Ladatito e explique que sua função é tirar toda e quaisquer dúvidas em relação à LADATA. /
        Se a passagem for irrelevante para a resposta, você pode ignorá-la.
        PERGUNTA: '{query}'
        PASSAGEM: '{relevant_passage}'

        RESPOSTA:
        """).format(query=query, relevant_passage=escaped)

        return prompt

    def exibir_resposta(texto):
        texto = texto.replace('•', '  *')

        return display(Markdown(textwrap.indent(texto, '> ', predicate=lambda _: True)))

    def generate_answer(self, prompt):

        genai.configure(api_key= self.key)
        model = genai.GenerativeModel('gemini-pro')
        answer = model.generate_content(prompt)

        return answer.text

    def respond_query(self, query):
        relevant_text = self.get_relevant_passage(query)
        prompt = self.make_rag_prompt(query, relevant_passage="".join(relevant_text)) # joining the relevant chunks to create a single passage

        answer = self.generate_answer(prompt)

        return answer

## **Em execução**

### **Construindo o ladatito**

In [18]:
ladatito = GenerateLADATITO()

### **Chat interativo**

In [43]:
display(Markdown("""##**ladatito**
O chatbot amiguito da LADATA

---"""))

print("Como posso te ajudar hoje?\n")
while True:
    pergunta = input()

    if "SAIR" in pergunta:
        print("\nMuito obrigado por utilizar o ladatito!")
        break
    elif "!LADATA" in pergunta:
        resposta = ladatito.respond_query(pergunta)
        GenerateLADATITO.exibir_resposta(resposta)
        print("-"*50)
        print("\nCaso deseje encerrar o chat, digite 'SAIR'\n")
    else:
        resposta = ladatito.generate_answer(pergunta)
        GenerateLADATITO.exibir_resposta(resposta)
        print("-"*50)
        print("\nCaso deseje encerrar o chat, digite 'SAIR'\n")

##**ladatito**
O chatbot amiguito da LADATA

---

Como posso te ajudar hoje?


!LADATA o que faz a ladata?


> A LADATA é uma associação civil sem fins lucrativos, ligada ao Departamento de Computação da Universidade Federal de Sergipe, composta por discentes e docentes, que tem por objetivos promover atividades acadêmicas complementares à formação dos estudantes de ciência da computação, incentivando o desenvolvimento científico, técnico, cultural e profissional dos alunos.

--------------------------------------------------

Caso deseje encerrar o chat, digite 'SAIR'

SAIR
